# Benchmarking for the api - version 0

### QFT + QFT^-1

#### Probamos con AerSimulator en la interfaz de qiskit

In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT

n = 10

qc = QuantumCircuit(n)

qc.x(0); qc.x(n-1); qc.x(n-2)

qc.append(QFT(n), range(n))

qc.append(QFT(n).inverse(), range(n))

qc.measure_all()

display(qc.draw())

qc = qc.decompose().decompose().decompose()

┌───┐┌──────┐┌───────┐ ░ ┌─┐                           
    q_0: ┤ X ├┤0     ├┤0      ├─░─┤M├───────────────────────────
         └───┘│      ││       │ ░ └╥┘┌─┐                        
    q_1: ─────┤1     ├┤1      ├─░──╫─┤M├────────────────────────
              │      ││       │ ░  ║ └╥┘┌─┐                     
    q_2: ─────┤2     ├┤2      ├─░──╫──╫─┤M├─────────────────────
              │      ││       │ ░  ║  ║ └╥┘┌─┐                  
    q_3: ─────┤3     ├┤3      ├─░──╫──╫──╫─┤M├──────────────────
              │      ││       │ ░  ║  ║  ║ └╥┘┌─┐               
    q_4: ─────┤4     ├┤4      ├─░──╫──╫──╫──╫─┤M├───────────────
              │  QFT ││  IQFT │ ░  ║  ║  ║  ║ └╥┘┌─┐            
    q_5: ─────┤5     ├┤5      ├─░──╫──╫──╫──╫──╫─┤M├────────────
              │      ││       │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         
    q_6: ─────┤6     ├┤6      ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────
              │      ││       │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
    q_7: ─────┤7     ├┤7      ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────
         ┌───┐│      ││       │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
    q_8: ┤ X ├┤8     ├┤8      ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
         ├───┤│      ││       │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
    q_9: ┤ X ├┤9     ├┤9      ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
         └───┘└──────┘└───────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 10/══════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                   0  1  2  3  4  5  6  7  8  9

Voy a probar con `qiskit`, usando `AerSimulator` de `qiskit_aer`.

In [2]:
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
backend = AerSimulator()
qct = transpile(qc, backend)
result = backend.run(qct, shots = 4000).result()
counts_qiskit_aer = result.get_counts()

for b, c in counts_qiskit_aer.items():
    print(f'{b}: {c}')


1100000001: 4000


Esto es evidente: al aplicar la `QFT` y la `IQFT` deberíamos obtener el valor de entrada.

#### Probamos con la api sin ruido, con una sola qpu

En primer lugar, cargamos lo módulos y las variables de entorno que necesitamos:

In [1]:
import os
import sys
import json
import time

install_path = os.getenv("INSTALL_PATH")
print(install_path)# como no me lo pilla pues lo pongo a mano
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation")
sys.path.insert(0, "/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/python")

/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation


In [2]:
from qpu import QPU, getQPUs
from qjob import gather

En primer lugar usamos la función `get_QPUs()` para crear nuestra lista de máquinas.

In [3]:
qpus = getQPUs()

print("QPUs disponibles:")
for q in qpus:
    print(f"QPU {q.id_}, backend: {q.backend.name}.")

QPUs disponibles:
QPU 0, backend: FakeQmio.


Vamos a obtener información sobre el backend de la primera QPU:

In [4]:
qpu0 = qpus[0]
qpu0.backend.set("coupling_map", [])
qpu0.backend.info()

--- Backend configuration ---
basis_gates: ['sx', 'x', 'rz', 'ecr']
conditional: True
coupling_map: []
custom_instructions: 
description: FakeQmio backend
gates: []
is_simulator: True
max_shots: 1000000
memory: True
n_qubits: 32
name: FakeQmio
simulator: AerSimulator
url: 
version: 


Probamos a mandar un `QJob` a `qpu0`, con el mismo número de shots.

In [8]:
qjob0 = qpu0.run(qc, shots = 4000, transpile = True)

In [9]:
result = qjob0.result()

In [10]:
from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts()).savefig("histograma_fakeqmio.png")

In [5]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(32,2); qc.h(30); qc.cx(30,31); qc.measure(30,0); qc.measure(31,1)

In [6]:
qjob0 = qpu0.run(qc, shots = 4000, transpile = False)

 {"config":{"shots": 4000, "method": "statevector", "memory_slots": 2}, "instructions":[{"name": "h", "qubits": [30], "params": []}, {"name": "cx", "qubits": [30, 31], "params": []}, {"name": "measure", "qubits": [30], "memory": [0]}, {"name": "measure", "qubits": [31], "memory": [1]}] }


In [9]:
result = qjob0.result()

In [10]:
from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts()).savefig("histograma_fakeqmio.png")

In [7]:
conf1 = {"config":{"shots": 4000, "method": "statevector", "memory_slots": 2}, "instructions":[{"name": "h", "qubits": [30], "params": []}, {"name": "cx", "qubits": [30, 31], "params": []}, {"name": "measure", "qubits": [30], "memory": [0]}, {"name": "measure", "qubits": [31], "memory": [1]}] }

In [8]:
conf1

{'config': {'shots': 4000, 'method': 'statevector', 'memory_slots': 2},
 'instructions': [{'name': 'h', 'qubits': [30], 'params': []},
  {'name': 'cx', 'qubits': [30, 31], 'params': []},
  {'name': 'measure', 'qubits': [30], 'memory': [0]},
  {'name': 'measure', 'qubits': [31], 'memory': [1]}]}

In [11]:
qjob0 = qpu0.run(qc, shots = 4000, transpile = True)

 {"config":{"shots": 4000, "method": "statevector", "memory_slots": 2}, "instructions":[{"name": "rz", "qubits": [30], "params": [1.5707963267948966]}, {"name": "sx", "qubits": [30], "params": []}, {"name": "rz", "qubits": [31], "params": [-3.141592653589793]}, {"name": "sx", "qubits": [31], "params": []}, {"name": "rz", "qubits": [31], "params": [-3.141592653589793]}, {"name": "ecr", "qubits": [30, 31], "params": []}, {"name": "x", "qubits": [30], "params": []}, {"name": "measure", "qubits": [30], "memory": [0]}, {"name": "measure", "qubits": [31], "memory": [1]}] }


In [12]:
conf2 = {"config":{"shots": 4000, "method": "statevector", "memory_slots": 2}, "instructions":[{"name": "rz", "qubits": [30], "params": [1.5707963267948966]}, {"name": "sx", "qubits": [30], "params": []}, {"name": "rz", "qubits": [31], "params": [-3.141592653589793]}, {"name": "sx", "qubits": [31], "params": []}, {"name": "rz", "qubits": [31], "params": [-3.141592653589793]}, {"name": "ecr", "qubits": [30, 31], "params": []}, {"name": "x", "qubits": [30], "params": []}, {"name": "measure", "qubits": [30], "memory": [0]}, {"name": "measure", "qubits": [31], "memory": [1]}] }

In [13]:
result = qjob0.result()
from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts()).savefig("histograma_fakeqmio_trans.png")